<a href="https://colab.research.google.com/github/alban12/llm-itu/blob/main/artifacts/Falcon_LLM_ITU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install the Transformers API from HF  

In [7]:
!pip install transformers datasets evaluate accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 

Install Pytorch

In [8]:
!pip install torch

ERROR: Operation cancelled by user


In [2]:
# !git clone https://github.com/alban12/llm-itu.git

Cloning into 'llm-itu'...
fatal: could not read Username for 'https://github.com': No such device or address


In [3]:
import pandas as pd

pd.read_csv("SampleSubmission.csv")

,Question_ID,Answer_ID
0,8138,0
1,8153,0
2,8156,0
3,8160,0
4,8165,0
...,...,...
2362,11996,0
2363,11997,0
2364,11998,0
2365,11999,0


In [ ]:
# Retrieve the questions in the training set (OK)
# Retrieve the data for RAG (OK)
# Package the data to make it easier to upload (TODO)
# Tokenize the training data to make it feedable to the model ( 1. I need to parse it as a json

Unzip the archive for RAG

In [7]:
!unrar x rel18.rar


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal

Unexpected end of archive

Extracting from rel18.rar

Creating    rel18                                                     OK
Extracting  rel18/21900-i10.docx                                           0%  OK 
Extracting  rel18/22011-i50.docx                                           0%  OK 
Extracting  rel18/22101-i50.docx                                           0%  OK 
Extracting  rel18/22104-i30.docx                                           0%  OK 
Extracting  rel18/22115-i00.docx                                           0%  OK 
Extracting  rel18/22125-i00.docx                                           0%  OK 
Extracting  rel18/22153-i20.docx                                           0%  OK 
Extracting  rel18/22179-i00.docx                                           0%  OK 
Extracting  rel18/22261-ic0.docx                              

In [7]:
category = json_data['question 0'].pop("category")
explanation = json_data['question 0'].pop("explanation")
answers = json_data['question 0'].pop("answer")
question_and_options = json_data['question 0']

Load the training dataset

In [12]:
import json
from datasets import Dataset

with open('TeleQnA_training.txt') as f:
    json_data = json.load(f)

category = []
explanation = []
answers = []
question_and_options = []
for key in json_data.keys():
  category.append(json_data[key].pop("category"))
  explanation.append(json_data[key].pop("explanation"))
  answers.append(json_data[key].pop("answer"))
  question_and_options.append(json_data[key])
my_dict = {"question_and_options": question_and_options,
           "category": category,
           "explanation": explanation,
           "answers": answers}
dataset = Dataset.from_dict(my_dict)

In [15]:
dataset

Dataset({
    features: ['question_and_options', 'category', 'explanation', 'answers'],
    num_rows: 1461
})

In [ ]:
# Boiler plate

# Call a pipeline on many items

pipe = pipeline("text-classification")
pipe(["This restaurant is awesome", "This restaurant is awful"])

# Iterate over a dataset

import datasets
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

pipe = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h", device=0)
dataset = datasets.load_dataset("superb", name="asr", split="test")

# KeyDataset (only *pt*) will simply return the item in the dict returned by the dataset item
# as we're not interested in the *target* part of the dataset. For sentence pair use KeyPairDataset
for out in tqdm(pipe(KeyDataset(dataset, "file"))):
    print(out)
    # {"text": "NUMBER TEN FRESH NELLY IS WAITING ON YOU GOOD NIGHT HUSBAND"}
    # {"text": ....}
    # ....

Initialize the Falcon model

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import transformers
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
   " TOMODIFY: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: *Squeaky chirruping*
Daniel: *Chirruping back*
Girafatron: *Bark*
Daniel: *Bark*
Girafatron: *Bark*
Daniel: *Bark*
Girafatron: *Chirrup*
Daniel: *Bark*
Girafatron: *Bark*
Daniel: *Bark*
Girafatron: *Bark*
Daniel: *Chirrup*
Girafatron: *Chirrup*
Daniel: 
